In [1]:
from vectorbtpro import *
import os

### Data Ingest 

In [2]:
'''
polygon_api_key = os.getenv('POLYGON_API_KEY')
vbt.PolygonData.set_custom_settings(
    client_config=dict(
        api_key=polygon_api_key
    )
)
data = vbt.PolygonData.pull(
    ["X:BTCUSD",
    "X:ETHUSD",
    "X:SOLUSD"],
    start="2019-01-01",
    end="2024-11-01",
    timeframe="1 day"
)

data.to_hdf('priceseries.h5')
'''

#data = vbt.HDFData.pull(['priceseries.h5/X:BTCUSD'])
data = vbt.HDFData.pull('priceseries.h5')




open_price = data.get('Open')
close_price = data.get('Close')
high_price = data.get('High')
low_price = data.get('Low')

### Indicator Parameter Space

In [3]:
wtypes = ["simple", "exp", "wilder"]
rsi_win = list(range(8, 21))
bbands_win = np.arange(10, 50, 5)
alphas = np.arange(1.5, 3.0, 0.5)

Bollinger-Band Calculation: $\alpha$ = number of standard deviations
$$
\begin{align*}
\text{Middle Band} &= \text{SMA}(\text{close}, n) \\
\text{Upper Band} &= \text{SMA}(\text{close}, n) + \alpha \times \sigma(\text{close}, n) \\
\text{Lower Band} &= \text{SMA}(\text{close}, n) - \alpha \times \sigma(\text{close}, n) \\\\
\text{Bandwidth} &= \frac{\text{Upper Band} - \text{Lower Band}}{\text{Middle Band}}
\end{align*}
$$

In [4]:
bbands = vbt.BBANDS.run(
    close_price,
    window=bbands_win,
    wtype=wtypes,
    alpha=alphas,
    param_product=True
)

### Trade Conditions

#### Bollinger Band Signal
<u>Sell signal</u>: Downward breakout while expanding  
If the daily low is below the lower band *AND* the bandwidth is above the lower bandwidth threshold.

<u>Buy signal</u>: Upward breakout while squeezing  
If the daily high is above the upper band *AND* the bandwidth is below the upper bandwidth threshold.

In [10]:
bandwidth = (bbands.upper - bbands.lower) / bbands.middle

entry_cond1 = data.get("Low").vbt < bbands.lower
entry_cond2 = bandwidth > 0.3
entry_cond3 = data.get("High").vbt > bbands.upper
entry_cond4 = bandwidth < 0.15

in_cond1, in_cond2, in_cond3, in_cond4 = vbt.pd_acc.x(entry_cond1, entry_cond2, entry_cond3, entry_cond4)
entry_signals = (in_cond1.vbt & in_cond2).vbt | (in_cond3.vbt & in_cond4)

exit_cond1 = data.get("High").vbt > bbands.upper
exit_cond2 = bandwidth > 0.3
exit_cond3 = data.get("Low").vbt < bbands.lower
exit_cond4 = bandwidth < 0.15

out_cond1, out_cond2, out_cond3, out_cond4 = vbt.pd_acc.x(exit_cond1, exit_cond2, exit_cond3, exit_cond4)
exit_signals = (out_cond1.vbt & out_cond2).vbt | (out_cond3.vbt & out_cond4)

# Caclulate the maximum number of exit signals after each entry signal
exit_signals.vbt.signals.pos_rank_after(entry_signals, reset_wait=0).max() + 1

bb_window  bb_wtype  bb_alpha  symbol  
10         simple    1.5       X:BTCUSD    17
                               X:ETHUSD    14
                               X:SOLUSD    26
                     2.0       X:BTCUSD    14
                               X:ETHUSD    12
                                           ..
45         wilder    2.0       X:ETHUSD    45
                               X:SOLUSD    48
                     2.5       X:BTCUSD     8
                               X:ETHUSD     3
                               X:SOLUSD    23
Length: 216, dtype: int64

#### Cleaning
Some signals shouln't be converted into orders. The maximum number of exit signals after each entry signal should be 1.

In [12]:
# select only the first signal (entry or exit), and ignore subsequent signals of the same type
entries, exits = entry_signals.vbt.signals.clean(exit_signals)

exits.vbt.signals.pos_rank_after(entries, reset_wait=0).max() + 1

bb_window  bb_wtype  bb_alpha  symbol  
10         simple    1.5       X:BTCUSD    1
                               X:ETHUSD    1
                               X:SOLUSD    1
                     2.0       X:BTCUSD    1
                               X:ETHUSD    1
                                          ..
45         wilder    2.0       X:ETHUSD    1
                               X:SOLUSD    1
                     2.5       X:BTCUSD    1
                               X:ETHUSD    1
                               X:SOLUSD    1
Length: 216, dtype: int64

### Create a Portfolio from Entry and Exit Conditions

In [13]:
pf = vbt.Portfolio.from_signals(
    close=close_price, 
    entries=entries, 
    exits=exits,
    size=100,
    size_type='value',
    init_cash='auto'
)

performance_metrics = [
    'max_dd', 
    'max_dd_duration',
    'sortino_ratio',
    'omega_ratio',
    'profit_factor',
    'expectancy',
    'total_orders',
    'win_rate',
    'avg_winning_trade_duration',
    'avg_losing_trade_duration'
]

stats_df = pf.stats(performance_metrics, agg_func=None)

#### Analysis
Which parameter combination yields the greatest Omega ratio (Risk-free rate $r_f = 0$)?
$$
\Omega(R, \tau) = \frac{\int_{\tau}^{\infty} [1 - F(R)] \, dR}{\int_{-\infty}^{\tau} F(R) \, dR}
$$

In [70]:
def get_wtype_stats(window_type, metric):
    win_mask = stats_df.index.get_level_values('bb_wtype') == window_type
    df = stats_df[win_mask]
    btc_mask = df.index.get_level_values('symbol') == 'X:BTCUSD'
    eth_mask = df.index.get_level_values('symbol') == 'X:ETHUSD'
    sol_mask = df.index.get_level_values('symbol') == 'X:SOLUSD'

    portfolio_data = pd.DataFrame({
        'BTCUSD': df[btc_mask][metric].values,
        'ETHUSD': df[eth_mask][metric].values,
        'SOLUSD': df[sol_mask][metric].values
    })

    return portfolio_data

In [71]:
pf_data = get_wtype_stats('exp', 'Omega Ratio')
param_comb = list(product(bbands_win, alphas))

pf_data.index = pd.MultiIndex.from_tuples(
    param_comb, 
    names=['bbands_win', 'bbands_alpha'])

pf_data['ETHUSD'].vbt.heatmap(x_level='bbands_alpha', y_level='bbands_win')


FigureWidget({
    'data': [{'colorscale': [[0.0, '#0d0887'], [0.1111111111111111, '#46039f'],
                             [0.2222222222222222, '#7201a8'], [0.3333333333333333,
                             '#9c179e'], [0.4444444444444444, '#bd3786'],
                             [0.5555555555555556, '#d8576b'], [0.6666666666666666,
                             '#ed7953'], [0.7777777777777778, '#fb9f3a'],
                             [0.8888888888888888, '#fdca26'], [1.0, '#f0f921']],
              'hoverongaps': False,
              'hovertemplate': 'bbands_alpha: %{x}<br>bbands_win: %{y}<br>value: %{z}<extra></extra>',
              'type': 'heatmap',
              'uid': '66013004-705d-40fe-ac04-a47aa82941bc',
              'x': array([1.5, 2. , 2.5]),
              'y': array([10, 15, 20, 25, 30, 35, 40, 45]),
              'z': array([[1.33362648, 1.26029291, 1.04539473],
                          [1.20394613, 1.281769  , 1.043356  ],
                          [1.17947907, 1.23493